# Sequential Q-learning

In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
import gym
import numpy as np

In [2]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

>**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [3]:
import gym

# Create the Cart-Pole game environment
#env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [4]:
env.reset()
for _ in range(10):
    # env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action) # take a random action
    print('state, action, reward, done, info:', state, action, reward, done, info)
    if done:
        env.reset()

state, action, reward, done, info: [-0.00454824 -0.20735203 -0.04576232  0.32077049] 0 1.0 False {}
state, action, reward, done, info: [-0.00869528 -0.01160926 -0.03934691  0.01401429] 1 1.0 False {}
state, action, reward, done, info: [-0.00892746  0.18405424 -0.03906663 -0.2908188 ] 1 1.0 False {}
state, action, reward, done, info: [-0.00524638 -0.01048952 -0.04488301 -0.01070849] 0 1.0 False {}
state, action, reward, done, info: [-0.00545617  0.18524642 -0.04509717 -0.31720789] 1 1.0 False {}
state, action, reward, done, info: [-0.00175124  0.38098072 -0.05144133 -0.62376491] 1 1.0 False {}
state, action, reward, done, info: [ 0.00586837  0.57678172 -0.06391663 -0.93219498] 1 1.0 False {}
state, action, reward, done, info: [ 0.01740401  0.7727052  -0.08256053 -1.24425878] 1 1.0 False {}
state, action, reward, done, info: [ 0.03285811  0.96878368 -0.10744571 -1.56161992] 1 1.0 False {}
state, action, reward, done, info: [ 0.05223378  1.16501446 -0.1386781  -1.88579697] 1 1.0 False {}


To shut the window showing the simulation, use `env.close()`.

If you ran the simulation above, we can look at the rewards:

In [5]:
# print(rewards[-20:])
# print(np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
# print(np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
# print(np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
# print(np.max(np.array(rewards)), np.min(np.array(rewards)))
# print(np.max(np.array(states)), np.min(np.array(states)))

The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [6]:
def model_input(state_size, lstm_size, batch_size=1):
    actions = tf.placeholder(tf.int32, [None], name='actions')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    labelQs = tf.placeholder(tf.float32, [None], name='labelQs')
        
    gru = tf.nn.rnn_cell.GRUCell(lstm_size)
    cell = tf.nn.rnn_cell.MultiRNNCell([gru], state_is_tuple=False)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return actions, states, targetQs, labelQs, cell, initial_state

In [7]:
# RNN generator or sequence generator
def generator(states, initial_state, cell, lstm_size, num_classes, reuse=False): 
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        inputs = tf.layers.dense(inputs=states, units=lstm_size)
        print(states.shape, inputs.shape)
        
        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        # dynamic means adapt to the batch_size
        inputs_rnn = tf.reshape(inputs, [1, -1, lstm_size]) # NxH -> 1xNxH
        print(inputs_rnn.shape, initial_state.shape)
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs_rnn, initial_state=initial_state)
        print(outputs_rnn.shape, final_state.shape)
        outputs = tf.reshape(outputs_rnn, [-1, lstm_size]) # 1xNxH -> NxH
        print(outputs.shape)

        # Last fully connected layer
        logits = tf.layers.dense(inputs=outputs, units=num_classes)
        print(logits.shape)
        #predictions = tf.nn.softmax(logits)
        
        # logits are the action logits
        return logits, final_state

In [8]:
def model_loss(action_size, hidden_size, states, cell, initial_state, actions, targetQs, labelQs):
    actions_logits, final_state = generator(states=states, cell=cell, initial_state=initial_state, 
                                            lstm_size=hidden_size, num_classes=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    Qs = tf.reduce_max(actions_logits*actions_labels, axis=1)
    lossQtgt = tf.reduce_mean(tf.square(Qs - targetQs)) # next state, next action and nextQs
    lossQlbl = tf.reduce_mean(tf.square(Qs - labelQs)) # current state, action, and currentQs
    lossQtgt_sigm = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs, 
                                                                           labels=tf.nn.sigmoid(targetQs)))
    lossQlbl_sigm = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs,
                                                                           labels=tf.nn.sigmoid(labelQs)))
    loss = lossQtgt + lossQlbl #+ lossQtgt_sigm + lossQlbl_sigm
    return actions_logits, final_state, loss, lossQtgt, lossQlbl, lossQtgt_sigm, lossQlbl_sigm

In [9]:
def model_opt(loss, learning_rate):
    """
    Get optimization operations in order
    :param loss: Generator loss Tensor for action prediction
    :param learning_rate: Learning Rate Placeholder
    :return: A tuple of (qfunction training, generator training, discriminator training)
    """
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]

    # # Optimize
    # with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
    # #opt = tf.train.AdamOptimizer(learning_rate).minimize(loss, var_list=g_vars)

    #grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(loss, g_vars), clip_norm=5) # usually around 1-5
    grads = tf.gradients(loss, g_vars)
    opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(grads, g_vars))

    return opt

In [10]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.actions, self.states, self.targetQs, self.labelQs, cell, self.initial_state = model_input(
            state_size=state_size, lstm_size=hidden_size)
        
        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.final_state, self.loss, self.lossQtgt, self.lossQlbl, self.lossQtgt_sigm, self.lossQlbl_sigm = model_loss(
            action_size=action_size, hidden_size=hidden_size, 
            states=self.states, actions=self.actions, 
            targetQs=self.targetQs, labelQs=self.labelQs, 
            cell=cell, initial_state=self.initial_state)

        # Update the model: backward pass and backprop
        self.opt = model_opt(loss=self.loss, learning_rate=learning_rate)

## Experience replay

Reinforcement learning algorithms can have stability issues due to correlations between states. To reduce correlations when training, we can store the agent's experiences and later draw a random mini-batch of those experiences to train on. 

Here, we'll create a `Memory` object that will store our experiences, our transitions $<s, a, r, s'>$. This memory will have a maxmium capacity, so we can keep newer experiences in memory while getting rid of older experiences. Then, we'll sample a random mini-batch of transitions $<s, a, r, s'>$ and train on those.

Below, I've implemented a `Memory` object. If you're unfamiliar with `deque`, this is a double-ended queue. You can think of it like a tube open on both sides. You can put objects in either side of the tube. But if it's full, adding anything more will push an object out the other side. This is a great data structure to use for the memory buffer.

In [11]:
from collections import deque

class Memory():    
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
        self.states = deque(maxlen=max_size)

## Exploration - Exploitation

To learn about the environment and rules of the game, the agent needs to explore by taking random actions. We'll do this by choosing a random action with some probability $\epsilon$ (epsilon).  That is, with some probability $\epsilon$ the agent will make a random action and with probability $1 - \epsilon$, the agent will choose an action from $Q(s,a)$. This is called an **$\epsilon$-greedy policy**.


At first, the agent needs to do a lot of exploring. Later when it has learned more, the agent can favor choosing actions based on what it has learned. This is called _exploitation_. We'll set it up so the agent is more likely to explore early in training, then more likely to exploit later in training.

## Q-Learning training algorithm

Putting all this together, we can list out the algorithm we'll use to train the network. We'll train the network in _episodes_. One *episode* is one simulation of the game. For this game, the goal is to keep the pole upright for 195 frames. So we can start a new episode once meeting that goal. The game ends if the pole tilts over too far, or if the cart moves too far the left or right. When a game ends, we'll start a new episode. Now, to train the agent:

* Initialize the memory $D$
* Initialize the action-value network $Q$ with random weights
* **For** episode = 1, $M$ **do**
  * **For** $t$, $T$ **do**
     * With probability $\epsilon$ select a random action $a_t$, otherwise select $a_t = \mathrm{argmax}_a Q(s,a)$
     * Execute action $a_t$ in simulator and observe reward $r_{t+1}$ and new state $s_{t+1}$
     * Store transition $<s_t, a_t, r_{t+1}, s_{t+1}>$ in memory $D$
     * Sample random mini-batch from $D$: $<s_j, a_j, r_j, s'_j>$
     * Set $\hat{Q}_j = r_j$ if the episode ends at $j+1$, otherwise set $\hat{Q}_j = r_j + \gamma \max_{a'}{Q(s'_j, a')}$
     * Make a gradient descent step with loss $(\hat{Q}_j - Q(s_j, a_j))^2$
  * **endfor**
* **endfor**

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [12]:
# print('state:', np.array(states).shape[1], 
#       'action size: {}'.format((np.max(np.array(actions)) - np.min(np.array(actions)))+1))

In [13]:
# Training parameters
# Network parameters
state_size = 4                 # number of units for the input state/observation -- simulation
action_size = 2                # number of units for the output actions -- simulation
hidden_size = 64               # number of units in each Q-network hidden layer -- simulation
batch_size = 32                # number of samples in the memory/ experience as mini-batch size
learning_rate = 0.001          # learning rate for adam

In [14]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

# Init the memory
memory = Memory(max_size=batch_size)

(?, 4) (?, 64)
(1, ?, 64) (1, 64)
(1, ?, 64) (1, 64)
(?, 64)
(?, 2)


## Populate the memory (exprience memory)

Here I'm re-initializing the simulation and pre-populating the memory. The agent is taking random actions and storing the transitions in memory. This will help the agent with exploring the game.

In [15]:
state = env.reset()
for _ in range(batch_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    state = next_state
    if done is True:
        state = env.reset()

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [16]:
memory.buffer[0]

[array([ 0.01734429,  0.0036899 , -0.0033179 ,  0.00365944]),
 1,
 array([ 0.01741809,  0.19885928, -0.00324471, -0.29006847]),
 1.0,
 0.0]

In [17]:
# states, rewards, actions

In [ ]:
# Now train with experiences
saver = tf.train.Saver() # save the trained model
rewards_list, loss_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    episode_reward = deque(maxlen=batch_size)
    
    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0
        loss_batch = []
        state = env.reset()
        initial_state = sess.run(model.initial_state) # Qs or current batch or states[:-1]

        # Training steps/batches
        while True:
            # Testing
            action_logits, final_state = sess.run([model.actions_logits, model.final_state],
                                                  feed_dict = {model.states: state.reshape([1, -1]), 
                                                               model.initial_state: initial_state})
            action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            memory.states.append([initial_state, final_state])
            total_reward += reward
            initial_state = final_state
            state = next_state

            # Training
            batch = memory.buffer
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            next_states = np.array([each[2] for each in batch])
            rewards = np.array([each[3] for each in batch])
            dones = np.array([each[4] for each in batch])
            rnn_states = memory.states
            initial_states = np.array([each[0] for each in rnn_states])
            final_states = np.array([each[1] for each in rnn_states])
            actions_logits = sess.run(model.actions_logits, 
                                      feed_dict = {model.states: states, 
                                                   model.initial_state: initial_states[0].reshape([1, -1])})
            labelQs = np.max(actions_logits, axis=1) # explore
            next_actions_logits = sess.run(model.actions_logits, 
                                           feed_dict = {model.states: next_states, 
                                                        model.initial_state: final_states[0].reshape([1, -1])})
            nextQs = np.max(next_actions_logits, axis=1) * (1-dones) # exploit
            targetQs = rewards + (0.99 * nextQs)
            loss, _, lossQlbl, lossQlbl_sigm, lossQtgt, lossQtgt_sigm = sess.run([model.loss, model.opt, 
                                                                                  model.lossQlbl, 
                                                                                  model.lossQlbl_sigm, 
                                                                                  model.lossQtgt, 
                                                                                  model.lossQtgt_sigm], 
                                            feed_dict = {model.states: states, 
                                                         model.actions: actions,
                                                         model.targetQs: targetQs,
                                                         model.labelQs: labelQs,
                                                         model.initial_state: initial_states[0].reshape([1, -1])})
            loss_batch.append(loss)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode: {}'.format(ep),
              'meanReward: {:.4f}'.format(np.mean(episode_reward)),
              'meanLoss: {:.4f}'.format(np.mean(loss_batch)))
        rewards_list.append([ep, np.mean(episode_reward)])
        loss_list.append([ep, np.mean(loss_batch)])
        if(np.mean(episode_reward) >= 500):
            break
    
    saver.save(sess, 'checkpoints/model5.ckpt')

Episode: 0 meanReward: 79.0000 meanLoss: 2.0324
Episode: 1 meanReward: 60.5000 meanLoss: 3.5357
Episode: 2 meanReward: 52.0000 meanLoss: 6.5075
Episode: 3 meanReward: 49.2500 meanLoss: 6.7281
Episode: 4 meanReward: 44.2000 meanLoss: 11.0835
Episode: 5 meanReward: 39.3333 meanLoss: 18.0686
Episode: 6 meanReward: 37.2857 meanLoss: 18.8232
Episode: 7 meanReward: 39.6250 meanLoss: 8.5163
Episode: 8 meanReward: 39.1111 meanLoss: 16.0877
Episode: 9 meanReward: 38.5000 meanLoss: 16.3416
Episode: 10 meanReward: 37.4545 meanLoss: 16.3686
Episode: 11 meanReward: 36.3333 meanLoss: 15.9999
Episode: 12 meanReward: 35.3846 meanLoss: 6.1176
Episode: 13 meanReward: 34.2143 meanLoss: 5.4468
Episode: 14 meanReward: 32.9333 meanLoss: 4.4587
Episode: 15 meanReward: 31.6250 meanLoss: 2.4320
Episode: 16 meanReward: 30.2941 meanLoss: 12.3756
Episode: 17 meanReward: 29.5000 meanLoss: 13.2213
Episode: 18 meanReward: 29.4211 meanLoss: 2.5846
Episode: 19 meanReward: 28.4000 meanLoss: 1.3671
Episode: 20 meanRewar

Episode: 163 meanReward: 190.1250 meanLoss: 19.7406
Episode: 164 meanReward: 199.2812 meanLoss: 7.0148
Episode: 165 meanReward: 203.8438 meanLoss: 22.8519
Episode: 166 meanReward: 215.9688 meanLoss: 4.7226
Episode: 167 meanReward: 226.9375 meanLoss: 3.1287
Episode: 168 meanReward: 222.7188 meanLoss: 192.2695
Episode: 169 meanReward: 228.7812 meanLoss: 10.2643
Episode: 170 meanReward: 237.4062 meanLoss: 29.7810
Episode: 171 meanReward: 248.0000 meanLoss: 64.6852
Episode: 172 meanReward: 257.4062 meanLoss: 16.4301
Episode: 173 meanReward: 272.3750 meanLoss: 13.1412
Episode: 174 meanReward: 268.1875 meanLoss: 28.8514
Episode: 175 meanReward: 278.9688 meanLoss: 11.6913
Episode: 176 meanReward: 285.6562 meanLoss: 1029.2108
Episode: 177 meanReward: 287.6875 meanLoss: 470.1177
Episode: 178 meanReward: 288.5625 meanLoss: 491.1910
Episode: 179 meanReward: 281.1250 meanLoss: 519.3581
Episode: 180 meanReward: 281.0000 meanLoss: 292.4215
Episode: 181 meanReward: 265.7500 meanLoss: 52.6490
Episode:

Episode: 325 meanReward: 146.4062 meanLoss: 1.8729
Episode: 326 meanReward: 147.1562 meanLoss: 1.8504
Episode: 327 meanReward: 149.2812 meanLoss: 3.3351
Episode: 328 meanReward: 149.9375 meanLoss: 1.9840
Episode: 329 meanReward: 152.0938 meanLoss: 5.0065
Episode: 330 meanReward: 166.3750 meanLoss: 4.7725
Episode: 331 meanReward: 181.1562 meanLoss: 7.6058
Episode: 332 meanReward: 196.0000 meanLoss: 13.8338
Episode: 333 meanReward: 209.8125 meanLoss: 16.9710
Episode: 334 meanReward: 224.6250 meanLoss: 18.2299
Episode: 335 meanReward: 230.7812 meanLoss: 26.3504
Episode: 336 meanReward: 229.6250 meanLoss: 22.3279
Episode: 337 meanReward: 214.9375 meanLoss: 337.4884
Episode: 338 meanReward: 214.9375 meanLoss: 9.4618
Episode: 339 meanReward: 213.7188 meanLoss: 89.6233
Episode: 340 meanReward: 199.8438 meanLoss: 12.6962
Episode: 341 meanReward: 199.2188 meanLoss: 16.6122
Episode: 342 meanReward: 199.2188 meanLoss: 3.3352
Episode: 343 meanReward: 211.5938 meanLoss: 17.0395
Episode: 344 meanRew

Episode: 483 meanReward: 204.3438 meanLoss: 42.5797
Episode: 484 meanReward: 208.7500 meanLoss: 11.0433
Episode: 485 meanReward: 206.8750 meanLoss: 38.7882
Episode: 486 meanReward: 204.5312 meanLoss: 190.6188
Episode: 487 meanReward: 200.9375 meanLoss: 232.6623
Episode: 488 meanReward: 199.3750 meanLoss: 36.3202
Episode: 489 meanReward: 200.6875 meanLoss: 27.4150
Episode: 490 meanReward: 210.1250 meanLoss: 2.2740
Episode: 491 meanReward: 204.2812 meanLoss: 49.5616
Episode: 492 meanReward: 205.3438 meanLoss: 4.6272
Episode: 493 meanReward: 205.3438 meanLoss: 9.3414
Episode: 494 meanReward: 218.5625 meanLoss: 20.5735
Episode: 495 meanReward: 218.5625 meanLoss: 23.8873
Episode: 496 meanReward: 218.9062 meanLoss: 19.7140
Episode: 497 meanReward: 234.1875 meanLoss: 19.1904
Episode: 498 meanReward: 230.2500 meanLoss: 303.4623
Episode: 499 meanReward: 229.9375 meanLoss: 488.0131
Episode: 500 meanReward: 232.4062 meanLoss: 87.7040
Episode: 501 meanReward: 228.5312 meanLoss: 100.9845
Episode: 5

Episode: 642 meanReward: 390.8750 meanLoss: 89.1836
Episode: 643 meanReward: 402.5625 meanLoss: 21.7524
Episode: 644 meanReward: 411.1562 meanLoss: 17.2782
Episode: 645 meanReward: 404.3750 meanLoss: 171.6169
Episode: 646 meanReward: 412.9375 meanLoss: 12.4713
Episode: 647 meanReward: 406.7500 meanLoss: 121.1025
Episode: 648 meanReward: 406.7500 meanLoss: 16.2702
Episode: 649 meanReward: 394.1562 meanLoss: 81.2229
Episode: 650 meanReward: 394.1562 meanLoss: 15.4357
Episode: 651 meanReward: 394.1562 meanLoss: 16.0780
Episode: 652 meanReward: 395.1250 meanLoss: 15.5929
Episode: 653 meanReward: 382.9688 meanLoss: 77.1804
Episode: 654 meanReward: 382.9688 meanLoss: 13.9353
Episode: 655 meanReward: 379.1562 meanLoss: 22.0176
Episode: 656 meanReward: 379.1562 meanLoss: 16.5671
Episode: 657 meanReward: 364.0312 meanLoss: 274.6700
Episode: 658 meanReward: 349.9375 meanLoss: 200.4227
Episode: 659 meanReward: 337.1562 meanLoss: 64.6522
Episode: 660 meanReward: 325.5625 meanLoss: 16.7361
Episode:

...

Episode: 4843 meanReward: 320.0000 meanLoss: 76.8808
Episode: 4844 meanReward: 323.5625 meanLoss: 62.0831
Episode: 4845 meanReward: 338.9062 meanLoss: 15.6097
Episode: 4846 meanReward: 338.9062 meanLoss: 18.0077
Episode: 4847 meanReward: 338.9062 meanLoss: 18.2645
Episode: 4848 meanReward: 338.9062 meanLoss: 17.6373
Episode: 4849 meanReward: 327.5625 meanLoss: 64.4414
Episode: 4850 meanReward: 327.5625 meanLoss: 16.1448
Episode: 4851 meanReward: 313.8438 meanLoss: 137.6349
Episode: 4852 meanReward: 313.8438 meanLoss: 15.8633
Episode: 4853 meanReward: 328.8438 meanLoss: 15.6901
Episode: 4854 meanReward: 344.0938 meanLoss: 16.1593
Episode: 4855 meanReward: 359.2188 meanLoss: 15.9171
Episode: 4856 meanReward: 372.2812 meanLoss: 16.5861
Episode: 4857 meanReward: 364.9375 meanLoss: 32.4325
Episode: 4858 meanReward: 364.9375 meanLoss: 7.0055
Episode: 4859 meanReward: 349.9062 meanLoss: 289.4898
Episode: 4860 meanReward: 349.8125 meanLoss: 431.0219
Episode: 4861 meanReward: 350.0625 meanLoss:

Episode: 4998 meanReward: 227.6875 meanLoss: 11.5148
Episode: 4999 meanReward: 237.8438 meanLoss: 14.6349
Episode: 5000 meanReward: 230.2812 meanLoss: 87.2492
Episode: 5001 meanReward: 241.4688 meanLoss: 9.5149
Episode: 5002 meanReward: 241.4688 meanLoss: 16.3893
Episode: 5003 meanReward: 231.9688 meanLoss: 236.9615
Episode: 5004 meanReward: 234.6875 meanLoss: 8.8941
Episode: 5005 meanReward: 232.8125 meanLoss: 88.9901
Episode: 5006 meanReward: 232.8125 meanLoss: 4.0663
Episode: 5007 meanReward: 239.5000 meanLoss: 25.5141
Episode: 5008 meanReward: 251.3438 meanLoss: 12.6626
Episode: 5009 meanReward: 248.7500 meanLoss: 262.8778
Episode: 5010 meanReward: 246.2500 meanLoss: 231.5988
Episode: 5011 meanReward: 240.7812 meanLoss: 314.8532
Episode: 5012 meanReward: 237.0000 meanLoss: 322.8435
Episode: 5013 meanReward: 223.1875 meanLoss: 86.6580
Episode: 5014 meanReward: 229.9062 meanLoss: 2.1059
Episode: 5015 meanReward: 244.3750 meanLoss: 12.4916
Episode: 5016 meanReward: 254.4688 meanLoss: 

Episode: 5153 meanReward: 330.3125 meanLoss: 200.6450
Episode: 5154 meanReward: 315.2500 meanLoss: 206.4226
Episode: 5155 meanReward: 305.0000 meanLoss: 25.0857
Episode: 5156 meanReward: 297.9062 meanLoss: 22.1009
Episode: 5157 meanReward: 291.0625 meanLoss: 25.9896
Episode: 5158 meanReward: 282.6875 meanLoss: 27.6995
Episode: 5159 meanReward: 280.0938 meanLoss: 13.7944
Episode: 5160 meanReward: 287.3750 meanLoss: 24.5908
Episode: 5161 meanReward: 302.5000 meanLoss: 11.7119
Episode: 5162 meanReward: 308.5938 meanLoss: 35.1170
Episode: 5163 meanReward: 323.8750 meanLoss: 14.1066
Episode: 5164 meanReward: 329.3125 meanLoss: 44.1131
Episode: 5165 meanReward: 317.5312 meanLoss: 56.5070
Episode: 5166 meanReward: 302.3750 meanLoss: 204.0614
Episode: 5167 meanReward: 287.2500 meanLoss: 284.9829
Episode: 5168 meanReward: 272.1562 meanLoss: 295.6091
Episode: 5169 meanReward: 259.7188 meanLoss: 89.8946
Episode: 5170 meanReward: 259.7188 meanLoss: 8.6996
Episode: 5171 meanReward: 259.7188 meanLos

Episode: 5309 meanReward: 117.4688 meanLoss: 11.5937
Episode: 5310 meanReward: 118.3750 meanLoss: 12.1589
Episode: 5311 meanReward: 119.9062 meanLoss: 11.1519
Episode: 5312 meanReward: 120.4375 meanLoss: 15.9819
Episode: 5313 meanReward: 123.6250 meanLoss: 12.2894
Episode: 5314 meanReward: 125.2812 meanLoss: 18.0774
Episode: 5315 meanReward: 137.6875 meanLoss: 9.8623
Episode: 5316 meanReward: 140.7812 meanLoss: 21.7858
Episode: 5317 meanReward: 137.8125 meanLoss: 219.9457
Episode: 5318 meanReward: 134.6562 meanLoss: 444.5276
Episode: 5319 meanReward: 131.5625 meanLoss: 539.1854
Episode: 5320 meanReward: 143.5000 meanLoss: 20.4312
Episode: 5321 meanReward: 140.6562 meanLoss: 154.5569
Episode: 5322 meanReward: 137.7812 meanLoss: 251.4688
Episode: 5323 meanReward: 134.8750 meanLoss: 259.7783
Episode: 5324 meanReward: 131.4375 meanLoss: 295.1402
Episode: 5325 meanReward: 128.1875 meanLoss: 299.5898
Episode: 5326 meanReward: 125.0312 meanLoss: 277.1880
Episode: 5327 meanReward: 121.4062 mea

Episode: 5464 meanReward: 304.3438 meanLoss: 8.3708
Episode: 5465 meanReward: 319.1562 meanLoss: 13.6487
Episode: 5466 meanReward: 334.2188 meanLoss: 17.3548
Episode: 5467 meanReward: 335.8125 meanLoss: 22.3367
Episode: 5468 meanReward: 341.9062 meanLoss: 13.2376
Episode: 5469 meanReward: 344.9062 meanLoss: 25.6462
Episode: 5470 meanReward: 338.9688 meanLoss: 314.5201
Episode: 5471 meanReward: 331.9062 meanLoss: 517.2267
Episode: 5472 meanReward: 316.6250 meanLoss: 522.5059
Episode: 5473 meanReward: 316.0938 meanLoss: 375.3187
Episode: 5474 meanReward: 317.2188 meanLoss: 98.4282
Episode: 5475 meanReward: 317.9688 meanLoss: 101.2343
Episode: 5476 meanReward: 312.6875 meanLoss: 13.5628
Episode: 5477 meanReward: 317.7500 meanLoss: 22.2900
Episode: 5478 meanReward: 312.9375 meanLoss: 22.6494
Episode: 5479 meanReward: 313.3438 meanLoss: 20.8682
Episode: 5480 meanReward: 316.5000 meanLoss: 19.7179
Episode: 5481 meanReward: 302.2188 meanLoss: 168.6230
Episode: 5482 meanReward: 305.0312 meanLo

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Average losses')

## Testing

Let's checkout how our trained agent plays the game.

In [184]:
import gym

# Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model-seq.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    initial_state = sess.run(model.initial_state) # Qs or current batch or states[:-1]
    state = env.reset()
    total_reward = 0
    while True:
        env.render()
        action_logits, initial_state = sess.run([model.actions_logits, model.final_state],
                                                feed_dict = {model.states: state.reshape([1, -1]), 
                                                             model.initial_state: initial_state})
        action = np.argmax(action_logits)
        state, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            break
print('total_reward:{}'.format(total_reward))
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


total_reward:120.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.